Простой пример: Простой Чат (Mediator как Сервер)
Пользователи чата общаются не напрямую, а через центральный сервер (Посредник), который рассылает сообщения всем остальным.



In [2]:
from abc import ABC, abstractmethod
from typing import List

# --- Mediator Interface (Интерфейс Посредника) ---
class ChatMediator(ABC):
    @abstractmethod
    def send_message(self, message: str, sender: 'User'):
        """Метод для отправки сообщения через посредника."""
        pass

    @abstractmethod
    def add_user(self, user: 'User'):
        """Метод для добавления пользователя в чат."""
        pass

# --- Colleague Interface/Class (Пользователь) ---
class User:
    """Базовый класс для пользователя (Коллега)."""
    def __init__(self, name: str, mediator: ChatMediator):
        self._name = name
        self._mediator = mediator

    @property
    def name(self) -> str:
        return self._name

    @abstractmethod
    def send(self, message: str):
        """Отправляет сообщение через посредника."""
        pass

    @abstractmethod
    def receive(self, message: str, sender_name: str):
        """Получает сообщение от посредника."""
        pass

# --- Concrete Mediator (Конкретный Посредник - Чат-Комната) ---
class ChatRoom(ChatMediator):
    """Реализация Посредника - управляет пользователями и сообщениями."""
    def __init__(self):
        self._users: List[User] = []
        print("ChatRoom: Initialized.")

    def add_user(self, user: User):
        print(f"ChatRoom: Adding user '{user.name}'.")
        self._users.append(user)

    def send_message(self, message: str, sender: User):
        print(f"ChatRoom: Message '{message}' received from '{sender.name}'. Broadcasting...")
        for user in self._users:
            # Сообщение не отправляется самому себе
            if user != sender:
                print(f"ChatRoom: Sending message to '{user.name}'.")
                user.receive(message, sender.name)

# --- Concrete Colleague (Конкретный Пользователь) ---
class ChatUser(User):
    """Конкретный пользователь чата."""
    def send(self, message: str):
        print(f"{self.name}: Sending message -> '{message}'")
        # Пользователь обращается к Посреднику для отправки
        self._mediator.send_message(message, self)

    def receive(self, message: str, sender_name: str):
        # Посредник вызывает этот метод у получателей
        print(f"{self.name}: Received message from '{sender_name}' <- '{message}'")

# --- Client Code ---
if __name__ == "__main__":
    # 1. Создаем Посредника
    chat_room = ChatRoom()

    # 2. Создаем Коллег, передавая им Посредника
    user1 = ChatUser("Alice", chat_room)
    user2 = ChatUser("Bob", chat_room)
    user3 = ChatUser("Charlie", chat_room)

    # 3. Регистрируем Коллег у Посредника
    chat_room.add_user(user1)
    chat_room.add_user(user2)
    chat_room.add_user(user3)

    print("\n--- Chat Session Starts ---")
    # 4. Коллеги начинают взаимодействовать через Посредника
    user1.send("Hi everyone!")
    print("-" * 20)
    user2.send("Hello Alice!")

# Вывод:
# ChatRoom: Initialized.
# ChatRoom: Adding user 'Alice'.
# ChatRoom: Adding user 'Bob'.
# ChatRoom: Adding user 'Charlie'.
#
# --- Chat Session Starts ---
# Alice: Sending message -> 'Hi everyone!'
# ChatRoom: Message 'Hi everyone!' received from 'Alice'. Broadcasting...
# ChatRoom: Sending message to 'Bob'.
# Bob: Received message from 'Alice' <- 'Hi everyone!'
# ChatRoom: Sending message to 'Charlie'.
# Charlie: Received message from 'Alice' <- 'Hi everyone!'
# --------------------
# Bob: Sending message -> 'Hello Alice!'
# ChatRoom: Message 'Hello Alice!' received from 'Bob'. Broadcasting...
# ChatRoom: Sending message to 'Alice'.
# Alice: Received message from 'Bob' <- 'Hello Alice!'
# ChatRoom: Sending message to 'Charlie'.
# Charlie: Received message from 'Bob' <- 'Hello Alice!'

ChatRoom: Initialized.
ChatRoom: Adding user 'Alice'.
ChatRoom: Adding user 'Bob'.
ChatRoom: Adding user 'Charlie'.

--- Chat Session Starts ---
Alice: Sending message -> 'Hi everyone!'
ChatRoom: Message 'Hi everyone!' received from 'Alice'. Broadcasting...
ChatRoom: Sending message to 'Bob'.
Bob: Received message from 'Alice' <- 'Hi everyone!'
ChatRoom: Sending message to 'Charlie'.
Charlie: Received message from 'Alice' <- 'Hi everyone!'
--------------------
Bob: Sending message -> 'Hello Alice!'
ChatRoom: Message 'Hello Alice!' received from 'Bob'. Broadcasting...
ChatRoom: Sending message to 'Alice'.
Alice: Received message from 'Bob' <- 'Hello Alice!'
ChatRoom: Sending message to 'Charlie'.
Charlie: Received message from 'Bob' <- 'Hello Alice!'


Сложный пример: Диалог с взаимозависимыми элементами GUI
Имитируем диалоговое окно аутентификации, где состояние кнопки "Login" зависит от состояния чекбокса "Agree to terms" и заполненности полей "Username" и "Password". Диалог выступает в роли Посредника.

Сложный пример: Диалог с взаимозависимыми элементами GUI
Имитируем диалоговое окно аутентификации, где состояние кнопки "Login" зависит от состояния чекбокса "Agree to terms" и заполненности полей "Username" и "Password". Диалог выступает в роли Посредника.

In [3]:
from abc import ABC, abstractmethod
from typing import Optional

# --- Mediator Interface ---
class DialogMediator(ABC):
    @abstractmethod
    def notify(self, sender: 'UIControl', event: str):
        """Компонент уведомляет Посредника о событии."""
        pass

# --- Colleague Base Class ---
class UIControl:
    """Базовый класс для элементов управления (Коллег)."""
    def __init__(self, dialog: DialogMediator):
        self._dialog = dialog # Каждый контрол знает своего Посредника

    def changed(self, event: str):
        """Уведомляет Посредника об изменении состояния."""
        print(f"  {type(self).__name__}: Notifying mediator about '{event}' event.")
        self._dialog.notify(self, event)

# --- Concrete Colleagues ---
class Button(UIControl):
    def __init__(self, name: str, dialog: DialogMediator):
        super().__init__(dialog)
        self.name = name
        self._is_enabled = False
        print(f"Button '{self.name}' created (disabled).")

    @property
    def enabled(self) -> bool:
        return self._is_enabled

    def set_enabled(self, enabled: bool):
        if self._is_enabled != enabled:
            self._is_enabled = enabled
            status = "enabled" if enabled else "disabled"
            print(f"  Button '{self.name}' is now {status}.")

    def click(self):
        print(f"Button '{self.name}': Clicked!")
        self.changed('click') # Уведомляем Посредника о клике

class TextBox(UIControl):
    def __init__(self, name: str, dialog: DialogMediator):
        super().__init__(dialog)
        self.name = name
        self._content = ""
        print(f"TextBox '{self.name}' created.")

    @property
    def content(self) -> str:
        return self._content

    def set_content(self, content: str):
        print(f"TextBox '{self.name}': Content changed to '{content}'.")
        if self._content != content:
            self._content = content
            self.changed('text_changed') # Уведомляем об изменении текста

class Checkbox(UIControl):
    def __init__(self, name: str, dialog: DialogMediator):
        super().__init__(dialog)
        self.name = name
        self._is_checked = False
        print(f"Checkbox '{self.name}' created (unchecked).")

    @property
    def checked(self) -> bool:
        return self._is_checked

    def set_checked(self, checked: bool):
        print(f"Checkbox '{self.name}': State changed to {checked}.")
        if self._is_checked != checked:
            self._is_checked = checked
            self.changed('check_changed') # Уведомляем об изменении состояния

# --- Concrete Mediator ---
class AuthenticationDialog(DialogMediator):
    """Посредник для диалога аутентификации."""
    def __init__(self):
        print("AuthenticationDialog: Initialized.")
        self.login_button: Optional[Button] = None
        self.cancel_button: Optional[Button] = None
        self.username_input: Optional[TextBox] = None
        self.password_input: Optional[TextBox] = None
        self.terms_checkbox: Optional[Checkbox] = None

    def notify(self, sender: UIControl, event: str):
        """Центральная точка обработки взаимодействий."""
        print(f"MEDIATOR: Received '{event}' from {type(sender).__name__} ('{sender.name}').")

        if isinstance(sender, Button) and event == 'click':
            if sender == self.login_button:
                self._attempt_login()
            elif sender == self.cancel_button:
                self._cancel_dialog()
        elif isinstance(sender, (TextBox, Checkbox)):
            # Если изменился любой из вводов или чекбокс, проверяем состояние кнопки Login
            self._check_login_button_state()

    def _check_login_button_state(self):
        """Проверяет условия и включает/выключает кнопку Login."""
        print("MEDIATOR: Checking conditions to enable Login button...")
        if (self.terms_checkbox and self.terms_checkbox.checked and
            self.username_input and self.username_input.content and
            self.password_input and self.password_input.content and
            self.login_button):
            print("MEDIATOR: Conditions met. Enabling Login button.")
            self.login_button.set_enabled(True)
        elif self.login_button:
            print("MEDIATOR: Conditions not met. Disabling Login button.")
            self.login_button.set_enabled(False)

    def _attempt_login(self):
        if self.login_button and self.login_button.enabled:
            username = self.username_input.content if self.username_input else ""
            password = self.password_input.content if self.password_input else ""
            print(f"\n*** MEDIATOR: Attempting login for user '{username}' with password '{'*' * len(password)}'... ***")
            # ... здесь логика реальной аутентификации ...
            print("*** MEDIATOR: Login successful (simulated)! ***")
        else:
            print("MEDIATOR: Login attempt failed (button disabled).")

    def _cancel_dialog(self):
        print("\n*** MEDIATOR: Cancel button clicked. Closing dialog (simulated)... ***")


# --- Client Code ---
if __name__ == "__main__":
    # 1. Создаем Посредника (Диалог)
    dialog = AuthenticationDialog()

    # 2. Создаем Коллег (Контролы), передавая им Посредника
    username = TextBox("Username", dialog)
    password = TextBox("Password", dialog)
    terms = Checkbox("AgreeTerms", dialog)
    login = Button("Login", dialog)
    cancel = Button("Cancel", dialog) # Cancel всегда включен

    # 3. Регистрируем Коллег у Посредника (устанавливаем ссылки в диалоге)
    dialog.username_input = username
    dialog.password_input = password
    dialog.terms_checkbox = terms
    dialog.login_button = login
    dialog.cancel_button = cancel
    cancel.set_enabled(True) # Кнопка Cancel всегда активна

    print("\n--- Simulating User Interaction ---")

    # Изначально кнопка Login выключена
    print(f"Login button enabled: {login.enabled}")

    # Пользователь вводит имя
    username.set_content("testuser")
    print(f"Login button enabled: {login.enabled}") # Всё еще выключена

    # Пользователь вводит пароль
    password.set_content("password123")
    print(f"Login button enabled: {login.enabled}") # Всё еще выключена

    # Пользователь ставит галочку
    terms.set_checked(True)
    print(f"Login button enabled: {login.enabled}") # Теперь должна включиться!

    # Пользователь кликает Login
    login.click()

    # Пользователь снимает галочку
    terms.set_checked(False)
    print(f"Login button enabled: {login.enabled}") # Должна выключиться

    # Пользователь кликает Cancel
    cancel.click()

# Вывод:
# AuthenticationDialog: Initialized.
# TextBox 'Username' created.
# TextBox 'Password' created.
# Checkbox 'AgreeTerms' created (unchecked).
# Button 'Login' created (disabled).
# Button 'Cancel' created (disabled).
#   Button 'Cancel' is now enabled.
#
# --- Simulating User Interaction ---
# Login button enabled: False
# TextBox 'Username': Content changed to 'testuser'.
#   TextBox: Notifying mediator about 'text_changed' event.
# MEDIATOR: Received 'text_changed' from TextBox ('Username').
# MEDIATOR: Checking conditions to enable Login button...
# MEDIATOR: Conditions not met. Disabling Login button.
# Login button enabled: False
# TextBox 'Password': Content changed to 'password123'.
#   TextBox: Notifying mediator about 'text_changed' event.
# MEDIATOR: Received 'text_changed' from TextBox ('Password').
# MEDIATOR: Checking conditions to enable Login button...
# MEDIATOR: Conditions not met. Disabling Login button.
# Login button enabled: False
# Checkbox 'AgreeTerms': State changed to True.
#   Checkbox: Notifying mediator about 'check_changed' event.
# MEDIATOR: Received 'check_changed' from Checkbox ('AgreeTerms').
# MEDIATOR: Checking conditions to enable Login button...
# MEDIATOR: Conditions met. Enabling Login button.
#   Button 'Login' is now enabled.
# Login button enabled: True
# Button 'Login': Clicked!
#   Button: Notifying mediator about 'click' event.
# MEDIATOR: Received 'click' from Button ('Login').
#
# *** MEDIATOR: Attempting login for user 'testuser' with password '***********'... ***
# *** MEDIATOR: Login successful (simulated)! ***
# Checkbox 'AgreeTerms': State changed to False.
#   Checkbox: Notifying mediator about 'check_changed' event.
# MEDIATOR: Received 'check_changed' from Checkbox ('AgreeTerms').
# MEDIATOR: Checking conditions to enable Login button...
# MEDIATOR: Conditions not met. Disabling Login button.
#   Button 'Login' is now disabled.
# Login button enabled: False
# Button 'Cancel': Clicked!
#   Button: Notifying mediator about 'click' event.
# MEDIATOR: Received 'click' from Button ('Cancel').
#
# *** MEDIATOR: Cancel button clicked. Closing dialog (simulated)... ***

AuthenticationDialog: Initialized.
TextBox 'Username' created.
TextBox 'Password' created.
Checkbox 'AgreeTerms' created (unchecked).
Button 'Login' created (disabled).
Button 'Cancel' created (disabled).
  Button 'Cancel' is now enabled.

--- Simulating User Interaction ---
Login button enabled: False
TextBox 'Username': Content changed to 'testuser'.
  TextBox: Notifying mediator about 'text_changed' event.
MEDIATOR: Received 'text_changed' from TextBox ('Username').
MEDIATOR: Checking conditions to enable Login button...
MEDIATOR: Conditions not met. Disabling Login button.
Login button enabled: False
TextBox 'Password': Content changed to 'password123'.
  TextBox: Notifying mediator about 'text_changed' event.
MEDIATOR: Received 'text_changed' from TextBox ('Password').
MEDIATOR: Checking conditions to enable Login button...
MEDIATOR: Conditions not met. Disabling Login button.
Login button enabled: False
Checkbox 'AgreeTerms': State changed to True.
  Checkbox: Notifying mediator 